In [16]:
! py -m pip install --upgrade pip
! pip install wheel setuptools pip --upgrade
! pip install swig
! pip install gymnasium[box2d]
! pip install moviepy
! pip install numpy

# Remarks

If the video does not work, it is advisable to restart the entire kernel after the initial installation.

In [6]:
import gymnasium
from gymnasium.wrappers.monitoring.video_recorder import VideoRecorder

env = gymnasium.make(
    "LunarLander-v2",
    continuous = False,
    gravity = -10.0,
    enable_wind = False,
    wind_power = 15.0,
    turbulence_power = 1.5,
    render_mode = "rgb_array"
)

observation, info = env.reset()
videoPath = "video.mp4"
video = VideoRecorder(env, videoPath)

for _ in range(200):
    env.render()
    video.capture_frame()
    action = env.action_space.sample()
    observation, reward, terminated, truncated, info = env.step(action)
    print(observation, reward, terminated, truncated, info)

video.close()
env.close()

[ 0.00535965  1.3985415   0.27724606 -0.2676438  -0.00531928 -0.04602603
  0.          0.        ] 0.2303596828672994 False False {}
[ 0.0081089   1.3919191   0.27725312 -0.29433706 -0.00761912 -0.04600114
  0.          0.        ] -1.469157581438651 False False {}
[ 0.01078529  1.3846952   0.26811248 -0.32106486 -0.00808425 -0.00930357
  0.          0.        ] -0.7494307532862468 False False {}
[ 0.01332998  1.3778809   0.25560245 -0.30285957 -0.00920672 -0.02245159
  0.          0.        ] 2.4656227508770714 False False {}
[ 0.01593475  1.3704715   0.2631488  -0.3293323  -0.0118397  -0.05266476
  0.          0.        ] -2.080122602163128 False False {}
[ 0.01846581  1.3624736   0.25387597 -0.3554643  -0.01260878 -0.01538287
  0.          0.        ] -0.8366187327633259 False False {}
[ 0.02105894  1.3538839   0.2616777  -0.38179663 -0.01494094 -0.0466475
  0.          0.        ] -2.0130263320860124 False False {}
[ 0.02365217  1.3446939   0.2616854  -0.40846542 -0.01727166 -0.046

Moviepy - Done !
Moviepy - video ready video.mp4


# Training the Lunar Landing

In [39]:
import numpy as np

observationSpaceLow = (-1.5, -1.5, -5., -5., -3.1415927, -5., -0., -0.)
observationSpaceHigh = (1.5, 1.5, 5., 5., 3.1415927, 5., 1., 1.)

observationSpace = list(zip(observationSpaceLow,observationSpaceHigh))

def makeIntervals(observationSpace, intervals=10):
    allStatesInterval = []
    
    for state in observationSpace[:-2]:
        stateInterval = []
        stepSize = (state[1] - state[0]) / intervals
        
        for step in np.arange(state[0], state[1], stepSize):
            stateInterval.append(np.round(step, decimals=1))
        
        allStatesInterval.append(stateInterval)
    
    return allStatesInterval

allStatesInterval =  makeIntervals(observationSpace)    

In [45]:
import itertools

# Define the lists of states
lists_of_states = [[-1.5, -1.2, -0.9, -0.6, -0.3, 0.0, 0.3, 0.6, 0.9, 1.2],
                   [-1.5, -1.2, -0.9, -0.6, -0.3, 0.0, 0.3, 0.6, 0.9, 1.2],
                   [-5.0, -4.0, -3.0, -2.0, -1.0, 0.0, 1.0, 2.0, 3.0, 4.0],
                   [-5.0, -4.0, -3.0, -2.0, -1.0, 0.0, 1.0, 2.0, 3.0, 4.0],
                   [-3.1, -2.5, -1.9, -1.3, -0.6, 0.0, 0.6, 1.3, 1.9, 2.5],
                   [-5.0, -4.0, -3.0, -2.0, -1.0, 0.0, 1.0, 2.0, 3.0, 4.0]]

permutations = list(itertools.product(*lists_of_states))

QTable = {}

for permutation in permutations:
    QTable[permutation] = np.zeros(4)

QTable[(-1.5, -1.5, -5.0, 4.0, 1.3, -3.0)]

array([0., 0., 0., 0.])